In [ ]:
%cd ..

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import collections
import glob
import functools
import logging
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import quantities as pq

import epych
from epych.statistics import alignment

In [ ]:
%matplotlib inline

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
CONDITIONS = ["go_gloexp", "go_seqctl", "lo_gloexp", "lonaive", "lo_rndctl", "igo_seqctl"]
PRETRIAL_SECONDS = 0.5
POSTTRIAL_SECONDS = 0.5

In [ ]:
NWB_SUBJECTS = glob.glob('/mnt/data/000253/sub-*/')

In [ ]:
PILOT_FILES = []

In [ ]:
NUM_TRIALS = 0
ODDBALL_ONSET = 0.
ODDBALL_OFFSET = 0.

In [ ]:
evoked = {}

In [ ]:
aligner = epych.statistics.alignment.AlignmentSummary.unpickle("/mnt/data/000253/visual_alignment")

In [ ]:
def visual_align(signal):
    return signal.select_channels(["VIS" in loc for loc in signal.channels.location]).median_filter()

In [ ]:
def samplings(cond):
    for s, subject_dir in enumerate(sorted(NWB_SUBJECTS)):
        subject = subject_dir.split('/')[-2]
        if not os.path.exists(subject_dir + "/" + cond):
            continue
        sampling = epych.recording.Sampling.unpickle(subject_dir + "/" + cond).smap(visual_align)
        global ODDBALL_ONSET
        global ODDBALL_OFFSET
        global NUM_TRIALS
        ODDBALL_ONSET += sampling.trials['stim3_start'].sum()
        ODDBALL_OFFSET += sampling.trials['stim3_end'].sum()
        NUM_TRIALS += len(sampling.trials)
        yield sampling
        del sampling
        logging.info("Loaded LFPs for %s in subject %s" % (cond, subject))

In [ ]:
summaries = {}

In [ ]:
def initialize_grandcat(key, signal):
    area = os.path.commonprefix([loc for loc in signal.channels.location])
    return epych.statistics.grand.GrandConcatenation(aligner.stats[area])

In [ ]:
for cond in CONDITIONS:
    summaries[cond] = epych.statistic.Summary(alignment.location_prefix, initialize_grand_avg)
    summaries[cond].calculate(samplings(cond))
    logging.info("Grand-averaged LFPs for condition %s" % cond)

In [ ]:
ODDBALL_ONSET /= NUM_TRIALS
ODDBALL_OFFSET /= NUM_TRIALS
events = {
    "Oddball Onset": (ODDBALL_ONSET, 'lightgreen'),
    "Oddball Offset": (ODDBALL_OFFSET, 'red'),
}

In [ ]:
avgs = {cond: {} for cond in CONDITIONS}

In [ ]:
for cond in CONDITIONS:
    avgs[cond] = epych.recording.EvokedSampling(epych.recording.empty_intervals(), epych.recording.empty_trials(), epych.recording.default_units(),
                                                **{k: v.result() for k, v in summaries[cond].stats.items()})
    avgs[cond].pickle("/mnt/data/000253/grandcat_%s" % cond)

In [ ]:
for cond in CONDITIONS:
    avgs[cond].baseline_correct(0, PRETRIAL_SECONDS).plot(vmin=-1e-4, vmax=1e-4, figure="mice_passiveglo_visual_%s_lfp.pdf" % cond, **events)

    csd = avgs[cond].smap(lambda sig: sig.downsample(4).current_source_density(depth_column="vertical"))
    csd.plot(figure="mice_passiveglo_visual_%s_csd.pdf" % cond, **events)
    logging.info("Plotted grand-averaged LFPs, CSDs for condition %s" % cond)